# Multi-agent Examples

In this example I show an example of an agent that writes a story using a series of steps.

1. The user provides an idea for their story.
2. The `Outline Bot` generates an outline for the story including section titles and descriptions.
3. The first section content is created by the `Story Bot` from section title/description and the overall story description.
4. The `Summary Bot` creates a summary of the newly generated chapter.
5. The section summary is passed to the `Story Bot` along with section title/description to generate the next section.
6. All sections follow steps 3-5.
...
7. The sections are combined into a full story.

![Story bot design diagram](https://storage.googleapis.com/public_data_09324832787/story_bot_design.svg)

In [ ]:
import pydantic
import typing

import sys
sys.path.append('../src/')

import simplechatbot
from simplechatbot.openai_agent import OpenAIAgent

First I create a new chatbot from OpenAI using the API stored in the keychain file.

I also create a new function `stream_it` that prints the result from the LLM as it is received and returns the full result of the LLM call.

In [ ]:
keychain = simplechatbot.APIKeyChain.from_json_file('../keys.json')
base_agent = OpenAIAgent.new(
    model_name = 'gpt-4o-mini', 
    api_key=keychain['openai'],
)

Next we need to direct the LLM to create an outline for the story. I do this by creating a system prompt describing the task of creating an outline and a pydantic class to direct the LLM on how to structure its response. This is important because the output of the outline bot will be used for generating chapters.

In [ ]:
class StoryOutline(pydantic.BaseModel):
    """Outline of the story."""

    story_topic: str = pydantic.Field(description="The topic of the story.")

    part1_title: str = pydantic.Field(description="Title of Part 1 of the story.")
    part1_description: str = pydantic.Field(description="Longer description of part 1.")

    part2_title: str = pydantic.Field(description="Title of Part 2 of the story.")
    part2_description: str = pydantic.Field(description="Longer description of part 2.")

    part3_title: str = pydantic.Field(description="Title of Part 3 of the story.")
    part3_description: str = pydantic.Field(description="Longer description of part 3.")

    def outline_str(self) -> str:
        return f'topic: {self.story_topic}\n\n{self.part1_title}: {self.part1_description}\n\n{self.part2_title}: {self.part2_description}\n\n{self.part3_title}: {self.part3_description}'


class OutlineBot:
    system_prompt = (
        "The user will provide you a description of a story, and you must create a chapter outline with titles and brief descriptions. "
        "Each section should contain a title and a brief description of what happens in that section. "
        "The sections should all be part of a single narrative ark, but each section should have a complete beginning, middle, and end. "
    )
    def __init__(self, base_agent: simplechatbot.Agent):
        self.agent = base_agent.new_agent_from_model(
            system_prompt=self.system_prompt,
        )

    def create_outline(self, story_description: str) -> StoryOutline:
        return self.agent.chat_structured(story_description, output_structure=StoryOutline).data

outline_bot = OutlineBot(base_agent)

q = f'Write an outline for a story about two friends who met when they were young and then lost touch. They meet again as adults and have to navigate their new relationship.'
outline: StoryOutline = outline_bot.create_outline(q)
print(outline.outline_str())

topic: Rekindling a Friendship

The Innocent Bond: In a small town, two children, Mia and Jake, form a deep bond during summer vacations. They share adventures, secrets, and dreams, making a pact to always stay friends. However, as they grow older, Mia moves away, and they lose touch. The chapter ends with Mia looking back at old photos, reminiscing about the carefree days of their friendship and wondering how Jake is doing.

A Chance Encounter: Years later, Mia returns to her hometown for a wedding. On the night of the event, she unexpectedly runs into Jake, who hasn't changed much but feels like a stranger. Their initial conversation is awkward, filled with nostalgia and uncertainty. But as the night unfolds, they find common ground in shared memories. They agree to grab coffee the next day to catch up, both feeling a mix of excitement and anxiety about seeing each other after so long.

Rebuilding the Past: During coffee, Mia and Jake navigate the complexities of their adult lives, r

Next we create a bot that will generate the actual chapter content based on information generated in the outline and a summary of the previous chapter (if one exists).

In [ ]:
class ChapterBot:
    system_prompt = (
        "You are designed to write a single chapter of a larger story based on the following information: \n\n"
        "+ Overall story topic: The topic of the full story.\n"
        "+ Chapter title: The title of the section you are writing.\n"
        "+ Chapter description: A longer description of what happens in the section.\n"
        "+ (optional) previous chapter summary: A summary of the previous chapter.\n"
        "Your responses should only include text that is part of the story. Do not include the chapter title \n"
        "or any other information that is not part of the story itself.\n"
        "The story chapter should be super short, so keep that in mind!"
    )

    def __init__(self, base_agent: simplechatbot.Agent):
        self.agent = base_agent.new_agent_from_model(
            system_prompt=self.system_prompt,
        )

    def write_chapter(
        self,
        story_topic: str,
        chapter_title: str,
        chapter_description: str,
        previous_chapter_summary: typing.Optional[str] = None,
    ) -> str:
        prompt = (
            f'General story topic: "{story_topic}"\n\n'
            f'Section title: "{chapter_title}"\n\n'
            f'Description: "{chapter_description}"\n\n'
            f'Previous chapter summary: "{previous_chapter_summary if previous_chapter_summary is not None else "No previous chapter - this is the first!"}"'
        )
        return self.agent.stream(prompt, add_to_history=False).print_and_collect().content

chapter_bot = ChapterBot(base_agent)

Now we actually create the chatper using the prompt.

In [5]:
chapter1 = chapter_bot.write_chapter(outline.story_topic, outline.part1_title, outline.part1_description)

The sun-drenched days of summer blended into a tapestry of laughter and adventure for Mia and Jake. Their small town, a canvas of vibrant colors and warm breezes, became the backdrop for their blossoming friendship. 

They often met at the old oak tree, its branches spreading wide like welcoming arms, where they shared whispered secrets and dreams of the future. One day, they painstakingly crafted a treasure map from an old cereal box, declaring their mission to find hidden gems in the nearby woods. With their imaginations running wild, they transformed fallen twigs into swords, prowled like pirates, and crowned each other as guardians of their secret world. 

Mia would sometimes pull out her worn notebook, filled with sketches of their adventures, while Jake would recount tales of daring knights and fearless dragons. They made a pact—an earnest promise to always stay friends—each believing their bond was unbreakable.

Then, as summer slipped away and the cold winds of fate began to bl

Next I create a bot that will summarize a chapter. We will eventually feed this into the creation of chapter 2.

In [ ]:
class SummaryBot:
    system_prompt = (
        'You need to create a summary of the story chapter provided to you by the user. '
        'The summary should include names of relevant characters and capture the story arc of the chapter. '
    )

    def __init__(self, base_agent: simplechatbot.Agent):
        self.agent = base_agent.new_agent_from_model(
            system_prompt=self.system_prompt,
        )

    def summarize(
        self,
        chapter_text: str,
    ) -> str:
        prompt = (
            f'Chapter text:\n\n{chapter_text}'
        )
        return self.agent.stream(prompt, add_to_history=False).print_and_collect().content

summary_bot = SummaryBot(base_agent)

In [7]:
ch1_summary = summary_bot.summarize(chapter1)

In this poignant chapter, we follow Mia and Jake, two friends who share a magical summer filled with adventures in their small town. Their friendship flourishes at the old oak tree, where they craft a treasure map and transform their imaginations into a world of pirates, knights, and dreams. They create a pact to always remain friends, believing their bond is unbreakable.

However, their idyllic summer comes to an abrupt end when Mia's family announces they are moving away, leading to a heart-wrenching farewell under the starlit sky. Mia and Jake hold hands tightly, each silently wishing that distance would not diminish their friendship.

As the years progress, Mia finds herself in a new, foreign place, making new friends but often reminiscing about her time with Jake. One evening, she discovers a box of memories, pulling out old photographs that remind her of their joyful days together. Reflecting on their shared adventures and the special bond they created, Mia wonders if Jake still 

Now we actually generate the second chapter using the outline and the previous chapter summary.

In [8]:
chapter2 = chapter_bot.write_chapter(outline.story_topic, outline.part2_title, outline.part2_description, ch1_summary)

Mia stepped out of the wedding venue, the soft glow of string lights illuminating the night as laughter and music floated through the air. She wrapped her shawl tighter around her shoulders, feeling a chill that had nothing to do with the evening breeze. It was a night of joy, yet a weight settled in her chest as she navigated the familiar streets of her hometown. 

Suddenly, she spotted him across the courtyard – Jake, leaning casually against a pillar, his tall frame and easy smile still as comforting as she remembered. The years melted into the background as their eyes locked, and it felt as if time had rewound to that summer when everything was simple. But beneath the familiarity lay an awkwardness; he was a stranger now, the miles between them stretching wider than she’d anticipated. 

“Jake?” Mia called out, her voice barely cutting through the thrum of the festivities. 

“Mia!” he responded, his surprise mirrored in his bright eyes. He stepped forward, hands shoved in his pocket

Create a summary for chapter 2 now.

In [9]:
ch2_summary = summary_bot.summarize(chapter2)

In this chapter, Mia finds herself stepping out of a wedding venue, overwhelmed by a mix of joy and nostalgia. As she walks through her hometown, she unexpectedly encounters Jake, her childhood friend and first love, who she hasn't seen in nearly a decade. Their initial interaction is filled with awkwardness and familiarity, highlighting the emotional distance that time and circumstances have created between them. 

They engage in small talk, reminiscing about their childhood adventures, such as creating a treasure map, which leads to laughter and a rekindling of their connection. Despite the years apart, they quickly find common ground, indicating an enduring bond beneath the surface. As Mia suggests meeting for coffee the next day, both characters sense a hopeful opportunity to explore what could be a revival of their relationship. The chapter concludes with Mia feeling a renewed sense of hope for the future as she watches Jake disappear back into the crowd, hinting at the possibilit

Use the summary of chapter 2 and outline information to create chapter 3.

In [10]:
chapter3 = chapter_bot.write_chapter(outline.story_topic, outline.part3_title, outline.part3_description, ch2_summary)

The coffee shop smelled of roasted beans and sweet pastries, a familiar scent that enveloped Mia as she entered, her heart racing with anticipation. She spotted Jake at a corner table, his back straight, absorbed in a steaming mug. Memories flooded her as she approached—of painting their childhood treehouse and sharing secrets under the stars.

“Hey,” she said, her voice barely above a whisper. 

He looked up, his expression brightening. “Hey, you made it.”

They exchanged warm smiles, feeling the shock of re-encounter fade, replaced by a comfortable, yet cautious atmosphere. As they sipped their coffees, the conversation flowed from light-hearted banter to deeper confessions. Mia shared tales of her career struggles, and Jake spoke of unpredictable travel adventures. 

“Remember the treasure map we made?” Mia chuckled, attempting to bridge any lingering gaps, her fingers tracing the rim of her cup.

Jake laughed, a sound that reminded her of so many summers spent laughing together. “Y

And now we can review the full text of the book.

In [11]:
story = f'''

Overview: {outline.story_topic}

== Chapter 1: {outline.part1_title} ==

{chapter1}


== Chapter 2: {outline.part2_title} ==

{chapter2}


== Chapter 3: {outline.part3_title} ==

{chapter3}

'''

import pprint
pprint.pprint(story)

('\n'
 '\n'
 'Overview: Rekindling a Friendship\n'
 '\n'
 '== Chapter 1: The Innocent Bond ==\n'
 '\n'
 'The sun-drenched days of summer blended into a tapestry of laughter and '
 'adventure for Mia and Jake. Their small town, a canvas of vibrant colors and '
 'warm breezes, became the backdrop for their blossoming friendship. \n'
 '\n'
 'They often met at the old oak tree, its branches spreading wide like '
 'welcoming arms, where they shared whispered secrets and dreams of the '
 'future. One day, they painstakingly crafted a treasure map from an old '
 'cereal box, declaring their mission to find hidden gems in the nearby woods. '
 'With their imaginations running wild, they transformed fallen twigs into '
 'swords, prowled like pirates, and crowned each other as guardians of their '
 'secret world. \n'
 '\n'
 'Mia would sometimes pull out her worn notebook, filled with sketches of '
 'their adventures, while Jake would recount tales of daring knights and '
 'fearless dragons. They 